In [1]:
#importing what is needed
import time
import pandas as pd
import requests
import json5

In [2]:
url_base = 'https://www.indeed.com/jobs?q&l=remote&fromage=7&limit=50&sort=date'

In [17]:
jobs = []

In [18]:
for pg in range(10000//50+1):
    err_count = 0
    r = requests.get(url_base+'&start={}'.format(pg*50))
    while r.status_code != 200 and err_count < 5:
        time.sleep(5)
        err_count += 1
        r = requests.get(url_base+'&start={}'.format(pg*50))
    if err_count == 5:
        continue
    text = r.text
    for i in range(50):
        start = text.find('jobmap[{}]'.format(i))
        if start==-1:
            continue
        end = text.find('jobmap[{}]'.format(i+1))
        s = text.find('{',start)
        if end == -1:
            e = text.find('}',s)
        else:
            e = text.rfind('}',s,end)
        job = json5.loads(text[s:e+1])
        jobs.append(job)
    time.sleep(5)

In [19]:
df = pd.DataFrame(jobs)

In [20]:
df.head()

,jk,efccid,srcid,cmpid,num,srcname,cmp,cmpesc,cmplnk,loc,country,zip,city,title,locid,rd
0,1d00f748ae6f6eea,caff23281376b83d,c6c40f68ed4ae948,132078c6fc4e9cec,0,Amazon.com,Amazon Support Services Costa Rica SRL,Amazon Support Services Costa Rica SRL,/q-Amazon-Support-Services-Costa-Rica-SRL-l-re...,Remote,US,,Remote,Work From Home 100% Virtual - Customer Service...,aaa2b906602aa8f5,0HcsNzDq-OLJ6bT4SC-8zAICdwM34F0uDTD1nxkaR7rzL3...
1,28646ed1f6749924,3e8562386cf03d35,62e6f9c57bf35405,11619ce0d3c2c733,1,Northrop Grumman,Northrop Grumman,Northrop Grumman,/q-Northrop-Grumman-l-remote-jobs.html,"Falls Church, VA 22042",US,,Falls Church,Recruiting Assistant,a81c27b9c065b72f,E4fhWX_aAFqF37QMCtQEyhxNNjPrsTulYVJeVzSI4sYKwV...
2,deda26e360538093,0011bf1afc70135e,80cbc2efb4bdc28e,68d50bffa07f3fd5,2,Sutherland,Sutherland,Sutherland,/q-Sutherland-l-remote-jobs.html,"Clifton, NJ 07015",US,07015,Clifton,Healthcare Customer Service Representative,62632718aaae6b97,E4fhWX_aAFqF37QMCtQEyhxNNjPrsTulYVJeVzSI4sa87h...
3,6eea105877eca085,7370623aab097544,f73baa5a09cd01e8,f7cf1bbbdf01bd07,3,The Mom Project,The Mom Project,The Mom Project,/q-The-Mom-Project-l-remote-jobs.html,Remote,US,,Remote,Marketing Researcher (100% Remote),aaa2b906602aa8f5,E4fhWX_aAFqF37QMCtQEyhxNNjPrsTulYVJeVzSI4sac32...
4,e849860c73fc0bd1,843da39184df95ce,149e66233ae4b346,5ce5af396d06487c,4,Indeed,MEIDE Singapore,MEIDE Singapore,/q-MEIDE-Singapore-l-remote-jobs.html,Remote,US,,Remote,Geylang/Jalan Besar Babysitting/Cleaning Job,aaa2b906602aa8f5,twcb2ZEJnz8JwcbwiR9wTeLAeaAvFtcuHRG0s9PBWc6vtV...


In [21]:
df.drop_duplicates(inplace=True)
df['job_link']='https://www.indeed.com/viewjob?jk='+df.jk
df.rename(columns={'cmp':'company','loc':'location'},inplace=True)
df = df[df['country']=="US"].copy()
df = df[['title','company','location','job_link']].copy()

In [22]:
df.head()

,title,company,location,job_link
0,Work From Home 100% Virtual - Customer Service...,Amazon Support Services Costa Rica SRL,Remote,https://www.indeed.com/viewjob?jk=1d00f748ae6f...
1,Recruiting Assistant,Northrop Grumman,"Falls Church, VA 22042",https://www.indeed.com/viewjob?jk=28646ed1f674...
2,Healthcare Customer Service Representative,Sutherland,"Clifton, NJ 07015",https://www.indeed.com/viewjob?jk=deda26e36053...
3,Marketing Researcher (100% Remote),The Mom Project,Remote,https://www.indeed.com/viewjob?jk=6eea105877ec...
4,Geylang/Jalan Besar Babysitting/Cleaning Job,MEIDE Singapore,Remote,https://www.indeed.com/viewjob?jk=e849860c73fc...


In [23]:
df.to_csv('../Data/indeed_remote_jobs.csv',index=False)